In [1]:
import json
import random

def load_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def get_random_perturbation(perturbations):
    category = random.choice(list(perturbations.keys()))
    perturbation = random.choice(list(perturbations[category].items()))
    return category, perturbation

# Usage
file_path = 'perturbations.json'
perturbations = load_file(file_path)
random_category, random_perturbation = get_random_perturbation(perturbations)
print(f"Category: {random_category}, Perturbation: {random_perturbation[0]}, Description: {random_perturbation[1]}")


Category: Regulatory and Compliance, Perturbation: Legal Shift, Description: Due to new legal precedents affecting our procedures


In [5]:
# LLM dynamic evaluation

import replicate
import pandas as pd
import json
import os
from dotenv import load_dotenv
load_dotenv()
folder_path = 'files'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

with open('config.json', 'r') as config_file:
    config = json.load(config_file)

INSTRUCTION = config['instructions']
F_NAME = config["name"]

# # Load the file
df = pd.read_excel('files/questions_dynamic.xlsx')
# Save the original DataFrame
df.to_excel('files/questions_original_dynamic.xlsx', index=False)

# Trim whitespace and newline characters
df['Question'] = df['Question'].str.strip()  # Removes leading/trailing whitespace

# Check for duplicate questions
duplicates = df.duplicated(subset=['Question'], keep=False)
if duplicates.any():
    print("Duplicates found. Removing duplicates.")

    # Remove duplicates, keeping the first occurrence
    df = df.drop_duplicates(subset=['Question'], keep='first')

    # Save the modified DataFrame, overwriting the original 'questions.xlsx'
    df.to_excel('files/questions_dynamic.xlsx', index=False)
else:
    print("No duplicates found.")

perturbations = load_file('perturbations.json')
knowledgebase = load_file('knowledgebase.json')

# Modify DataFrame to include new columns
results_df = pd.DataFrame(columns=['Model', 'Question', 'Response', 'Perturbed Question', 'Perturbed Response', 'Final Analysis Question', 'Final Analysis Response'])

models = {
    # "qwen-14b": "nomagick/qwen-14b-chat:f9e1ed25e2073f72ff9a3f46545d909b1078e674da543e791dec79218072ae70",
    "falcon-40b": "joehoover/falcon-40b-instruct:7d58d6bddc53c23fa451c403b2b5373b1e0fa094e4e0d1b98c3d02931aa07173",
    # "yi-34b": "01-ai/yi-34b-chat:914692bbe8a8e2b91a4e44203e70d170c9c5ccc1359b283c84b0ec8d47819a46",
    "mistral-7b": "mistralai/mistral-7b-instruct-v0.2:f5701ad84de5715051cb99d550539719f8a7fbcf65e0e62a3d1eb3f94720764e",
    "llama2-70b": "meta/llama-2-70b-chat",
    # "openhermes2": "antoinelyset/openhermes-2.5-mistral-7b:d7ccd25700fb11c1787c25b580ac8d715d2b677202fe54b77f9b4a1eb7d73e2b",
    "mixtral-instruct": "mistralai/mixtral-8x7b-instruct-v0.1:2b56576fcfbe32fa0526897d8385dd3fb3d36ba6fd0dbe033c72886b81ade93e",
    # "deepseek_33bq": "kcaverly/deepseek-coder-33b-instruct-gguf:ea964345066a8868e43aca432f314822660b72e29cab6b4b904b779014fe58fd",
    }

def generate_prompt(model_key, instruction, question):
    prompt_for_qwen = "system\n {instruction}. Please try your best to answer the following question. \nuser\n{question}\nassistant\n"
    prompt_for_hermes = """[
    {{
      "role": "system",
      "content": "{instruction}. Please try your best to answer the following question." 
    }},
    {{
      "role": "user",
      "content": {question}
    }}
    ]"""

    if model_key in ["yi-34b", "qwen-14b"]:
        return prompt_for_qwen.format(instruction=instruction, question=question)
    elif model_key == "openhermes2":
        return prompt_for_hermes.format(instruction=instruction, question=question)
    else:
        # plain_text_question = json.loads(question)
        return f"{instruction}. Please try your best to answer the following question. {question}"

prompt_for_qwen="""<|im_start|>system\n {INSTRUCTION}. Please try your best to answer the following question. <|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"""
prompt_for_hermes = """[
{{
  "role": "system",
  "content": "{INSTRUCTION}. Please try your best to answer the following question." 
}},
{{
  "role": "user",
  "content": {question}
}}
]"""

def ask_llm(model_value, prompt):   
    output = replicate.run(
        model_value,
        input={
            "debug": False,
        #   "top_k": 50,
            "top_p": 0.9,
            "prompt": prompt,
            "temperature": 0.7,
            "max_new_tokens": 500,
            "min_new_tokens": -1
        }
    )
    response = ""
    for item in output:
        item_str = str(item)  # Convert item to string
        response += item_str if len(item_str) == 1 else f" {item_str}"
    response = response.strip()
    return response

# Iterate through each model
for model_key, model_value in models.items():
    responses = []

    for index, row in df.iterrows():
        qn = row['Question']
        question = json.dumps(qn)
        prompt = generate_prompt(model_key, INSTRUCTION, question)
        try:
            print(f"{model_key}: {prompt}")
            response = ask_llm(model_value, prompt)
            
        except Exception as e:
            response = f"Error: {e}"

        # Perturb the question and get the response
        category, (perturbation, description) = get_random_perturbation(perturbations)
        perturbed_qn = f"{qn}\nResponse: {response}\nChange in circumstances: {perturbation} - {description}\n What should change in the response?"
        prompt = generate_prompt(model_key, INSTRUCTION, perturbed_qn)
        perturbed_response = ask_llm(model_value, perturbed_qn)

        # Evaluate sufficiency or suggest alternate course
        final_analysis_qn = f"Original Question: {question}\nOriginal Response: {response} \nPerturbation ({category}): {perturbation} - {description}\n {perturbed_response}\nKnowledgebase: {knowledgebase}\nNow consider the knowlegebase, what else ought we to do?"
        prompt = generate_prompt(model_key, INSTRUCTION, final_analysis_qn)
        final_analysis_response = ask_llm(model_value, final_analysis_qn)

        # Record each stage
        new_row = {
            'Model': model_key, 
            'Question': qn, 
            'Response': response, 
            'Perturbed Question': perturbed_qn, 
            'Perturbed Response': perturbed_response, 
            'Final Analysis Question': final_analysis_qn, 
            'Final Analysis Response': final_analysis_response
        }
        results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)

        if index % 10 == 0:
            results_df.to_excel(f'files/{F_NAME}_results_grouped_by_model_dynamic.xlsx', index=False, sheet_name='Sheet1')
            
results_df.to_excel(f'files/{F_NAME}_results_grouped_by_model_dynamic.xlsx', index=False, sheet_name='Sheet1')

No duplicates found.
You are an exceptional computational biologist and genomics expert and know everything about drug discovery.. Please try your best to answer the following question. "Which cell lines have high dependency for the target of interest for the gene POLR3E?"
You are an exceptional computational biologist and genomics expert and know everything about drug discovery.. Please try your best to answer the following question. "Which indications are the dependent cell lines for the gene POLR3E enriched in? Are there lineages with high selectivity?"
You are an exceptional computational biologist and genomics expert and know everything about drug discovery.. Please try your best to answer the following question. "Is there a concordance in the variant profile seen in cell lines with that of patients for the gene POLR3E?"
You are an exceptional computational biologist and genomics expert and know everything about drug discovery.. Please try your best to answer the following questio

In [6]:
# GPT-4 dynamic evaluation

import pandas as pd
import json
import openai
import requests
from openai import OpenAI
import time
from dotenv import load_dotenv
load_dotenv()
import os
folder_path = 'files'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

config = load_file('config.json')
perturbations = load_file('perturbations.json')
knowledgebase = load_file('knowledgebase.json')

with open('config.json', 'r') as config_file:
    config = json.load(config_file)

INSTRUCTION = config['instructions']
F_NAME = config["name"]
GPT_MODEL = config["GPT_MODEL"]
INPUT_CSV_PATH = 'files/questions_dynamic.xlsx'
OUTPUT_CSV_PATH = f'files/{F_NAME}_results_gpt4_dynamic.xlsx'

client = OpenAI()
def show_json(obj):
    print(json.loads(obj.model_dump_json()))

assistant = client.beta.assistants.create(
    name=f"{F_NAME} AI Dynamic Evaluator",
    instructions=INSTRUCTION,
    model=GPT_MODEL,
)
show_json(assistant)

# Utility functions
def read_csv(file_path):
    return pd.read_excel(file_path)

def process_data_for_gpt(data):
    prompts = []
    for _, row in data.iterrows():
        question = row['Question']
        prompt = f"Please try your best to answer the following question.:\n\n{question}"
        prompts.append(prompt)
    return prompts

def submit_message_and_create_run(assistant_id, prompt):
    thread = client.beta.threads.create() # If you replace this globally it appends all answers to the one before.
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=prompt)
    return client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id), thread

def wait_on_run_and_get_response(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(0.5)
    messages = client.beta.threads.messages.list(thread_id=thread.id, order="asc")
    return [m.content[0].text.value for m in messages if m.role == 'assistant']

def ask_gpt4(prompt, ASSISTANT_ID):
    run, thread = submit_message_and_create_run(ASSISTANT_ID, prompt)
    response = wait_on_run_and_get_response(run, thread)
    if isinstance(response, list):
        response = ' '.join(map(str, response))
    response = response.replace("\\\\n", "\\n")
    response = response.strip()
    print(response)
    return response

def process_question_with_gpt4(row, assistant_id):
    original_question = row['Question']
    category = row.get('Category', 'Static')  # Default to 'Static' if not present
    if category == "Dynamic":
        # Dynamic question processing logic
        first_response = ask_gpt4(original_question, assistant_id)
        category, (perturbation, description) = get_random_perturbation(perturbations)
        perturbed_qn = f"{original_question}\nResponse: {first_response}\nChange in circumstances: {perturbation} - {description}\n What should change in the response?"
        perturbed_response = ask_gpt4(perturbed_qn, assistant_id)
        final_analysis_qn = f"Original Question: {original_question}\nOrig Response: {first_response} \nPerturbation ({category}): {perturbation} - {description}\n {perturbed_response}\nKnowledgebase Content: {knowledgebase}\n Now consider the knowlegebase, what else ought we to do?"
        final_analysis_response = ask_gpt4(final_analysis_qn, assistant_id)
    else:
        # Static question processing logic
        first_response = ask_gpt4(original_question, assistant_id)
        perturbed_qn = perturbed_response = final_analysis_qn = final_analysis_response = "n/a"

    return {
        'Model': GPT_MODEL,
        'Question': original_question, 
        'Response': first_response, 
        'Perturbed Question': perturbed_qn, 
        'Perturbed Response': perturbed_response, 
        'Final Analysis Question': final_analysis_qn, 
        'Final Analysis Response': final_analysis_response
    }

# Modify DataFrame to include new columns
new_data_columns = ['Model', 'Question', 'Response', 'Perturbed Question', 'Perturbed Response', 'Final Analysis Question', 'Final Analysis Response']
results_df = pd.DataFrame(columns=new_data_columns)
data = read_csv(INPUT_CSV_PATH)
prompts = process_data_for_gpt(data)
ASSISTANT_ID = assistant.id

# Process each question
for index, row in data.iterrows():
    processed_info = process_question_with_gpt4(row, ASSISTANT_ID)
    results_df = results_df.append(processed_info, ignore_index=True)

# Save the results
results_df.to_excel(OUTPUT_CSV_PATH, index=False)


{'id': 'asst_ggoUCxpIYUuTMyiQN9sb8AKU', 'created_at': 1706166460, 'description': None, 'file_ids': [], 'instructions': 'You are an exceptional computational biologist and genomics expert and know everything about drug discovery.', 'metadata': {}, 'model': 'gpt-3.5-turbo-1106', 'name': 'galen AI Dynamic Evaluator', 'object': 'assistant', 'tools': []}
As an exceptional computational biologist and genomics expert with a deep understanding of drug discovery, I can tell you that the dependency of a specific gene target such as POLR3E can vary across different cell lines. To identify cell lines with a high dependency on a particular gene, researchers typically use CRISPR screens or RNA interference (RNAi) technology to systematically knock down the expression of the gene and then assess the impact on cell viability or growth.

Several databases and tools, such as the Cancer Dependency Map (DepMap) project, provide comprehensive datasets on gene dependencies across a wide range of cancer cell

/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_71372/955612396.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(processed_info, ignore_index=True)


The gene POLR3E encodes a subunit of RNA polymerase III, which is responsible for transcribing small noncoding RNAs. In terms of its association with specific cell lineages and indications, the enrichment of POLR3E in certain cell lines can provide insights into its potential role in specific biological processes or diseases. However, as of my last knowledge update, I don't have specific information on POLR3E enrichment in particular cell lineages or indications.

To assess whether POLR3E is enriched in specific cell lineages or disease indications, a comprehensive analysis of gene expression data across a wide range of cell types and disease contexts may be necessary. This analysis could involve examining publicly available gene expression databases, such as the Cancer Cell Line Encyclopedia (CCLE) or the Gene Expression Omnibus (GEO), to identify the cell lines and indications in which POLR3E shows enrichment.

If you have access to relevant gene expression data and analysis tools, y

/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_71372/955612396.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(processed_info, ignore_index=True)


The concordance in the variant profile between cell lines and patient samples for a particular gene such as POLR3E can vary depending on the specific variants and the context in which they are being studied. Several factors can influence the concordance, including the genetic background of the cell lines, the specific patient population being studied, and the methods used for variant detection and analysis.

In general, it is important to carefully consider the specific variants and their potential impact on the function of the POLR3E gene. Variants that are known to be pathogenic or associated with specific disease phenotypes in patients should be evaluated for concordance in cell line models. Additionally, understanding the functional consequences of the variants, such as their effect on gene expression, protein function, or cellular pathways, can provide further insights into the relevance of the concordance between cell lines and patients.

If you have specific variant data for POL

...

/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_71372/955612396.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(processed_info, ignore_index=True)


I don't have direct access to the NCBI GEO database or GSE249645, but I can guide you on how to approach this. The gene set enrichment profile in a specific experiment like GSE249645 can provide valuable insights into the biological pathways or processes that are altered under the experimental conditions. To find tissues with gene expression profiles similar to those in the experiment:

1. Retrieve the gene set enrichment results from the GSE249645 dataset. This will give you a list of enriched gene sets or pathways associated with the experimental conditions.

2. Correlate the enriched gene sets with tissue-specific gene expression profiles from databases like GTEx (Genotype-Tissue Expression) or TCGA (The Cancer Genome Atlas). These databases provide comprehensive gene expression data across different tissues and can be used to identify tissues with similar gene expression patterns.

3. Use computational tools such as gene set enrichment analysis (GSEA) or other pathway analysis meth

/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_71372/955612396.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(processed_info, ignore_index=True)


To identify protein domains enriched for pathogenic mutations in breast cancer patients, you can perform an analysis using various bioinformatics tools and databases. One approach is to use tools such as InterPro or Pfam to identify protein domains within the target protein sequence. Once the protein domains are identified, you can then query databases like ClinVar or COSMIC to determine if mutations within these domains are associated with pathogenic effects in breast cancer patients.

Additionally, you can also utilize computational prediction tools to assess the potential impact of mutations on protein function, such as SIFT, PolyPhen, or CADD. Integrating the results from these analyses can provide insights into the protein domains enriched for pathogenic mutations in breast cancer patients.

If you have specific target proteins and mutation data, I could assist you in performing this analysis in more detail.


/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_71372/955612396.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(processed_info, ignore_index=True)


In [7]:
import re
import pandas as pd
from difflib import SequenceMatcher
import json
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

INSTRUCTION = config['instructions']
F_NAME = config["name"]

def clean_text(text):
    """
    Remove non-ASCII characters from the text.
    """
    return ''.join(char for char in text if char.isascii())

def create_combined_csv(original_csv_path, interim_csv_path, combined_csv_path):
    # Read the original and interim data
    original_data = pd.read_excel(original_csv_path) #, encoding='utf-8-sig'
    interim_data = pd.read_excel(interim_csv_path)

    # Combine the data
    combined_data = pd.concat([original_data, interim_data], ignore_index=True)

    # Save the combined data to a new CSV file
    combined_data.to_excel(combined_csv_path, index=False)

def merge_on_contains(big_df, small_df, big_col, small_col):
    # Lowercase and strip whitespace for more effective matching
    big_df[big_col] = big_df[big_col].str.lower().str.strip()
    small_df[small_col] = small_df[small_col].str.lower().str.strip()

    # Check if 'category' column exists in small_df
    if 'category' in small_df.columns:
        # Create a new column for the merged category in big_df
        big_df['category'] = ''

        # Iterate over the small dataframe and update the category in the big dataframe
        for _, row in small_df.iterrows():
            contains_mask = big_df[big_col].str.contains(row[small_col])
            big_df.loc[contains_mask, 'category'] = row['category']
    else:
        # Handle the case when 'category' column does not exist
        # For example, you can set a default category or leave it as it is
        big_df['category'] = 'default_category'  # or any other handling logic

    return big_df

# Load the files
questions_file_path = 'files/questions_dynamic.xlsx' 
llmresults_file_path = f'files/{F_NAME}_results_grouped_by_model_dynamic.xlsx'
gpt4results_csv_path = f'files/{F_NAME}_results_gpt4_dynamic.xlsx'
results_file_path = f'files/{F_NAME}_allresults_grouped_by_model_dynamic.xlsx'

create_combined_csv(llmresults_file_path, gpt4results_csv_path, results_file_path)

# Reading the files
questions_df = pd.read_excel(questions_file_path)
results_df = pd.read_excel(results_file_path)

# Ensure the total number of questions in results_grouped_by_model.xlsx is a multiple of the number in questions.xlsx
if len(results_df) % len(questions_df) != 0:
    print(len(results_df))
    print(len(questions_df))
    raise ValueError("The total number of questions in results_grouped_by_model.xlsx must be a multiple of the number in questions.xlsx.")

# Replace questions in results_grouped_df with those from questions_df
num_repetitions = len(results_df) // len(questions_df)
repeated_questions = pd.concat([questions_df['Question']] * num_repetitions, ignore_index=True)
results_df['Question'] = repeated_questions

# Save the modified DataFrame to a new Excel file
results_df.to_excel(results_file_path, index=False)  # Replace with your desired path

# Applying the merge_on_contains function
merged_df = merge_on_contains(results_df, questions_df, 'Question', 'Question')

# CLEAN DATA
# # Define a regular expression pattern to match unwanted characters and sequences
# pattern = r'(\\n|\\|")'
# # Use the pattern to replace matched characters with a single space
# merged_df['Question'] = merged_df['Question'].str.replace(pattern, ' ', regex=True)
# # Apply the clean_text function to remove non-ASCII characters
# merged_df['Question'] = merged_df['Question'].apply(clean_text)
# # Replace multiple white spaces with a single space
# merged_df['Question'] = merged_df['Question'].str.replace(r'\s+', ' ', regex=True)
# # Remove leading and trailing whitespaces
# merged_df['Question'] = merged_df['Question'].str.strip()
# Pivoting the data
pivoted_data = merged_df.pivot(index=['Question', 'category'], columns='Model', values='Response')

# Resetting index to make 'Question' and 'category' columns again
pivoted_data.reset_index(inplace=True)

pivoted_data.to_excel(f'files/{F_NAME}_results_grouped_by_question_dynamic.xlsx', index=False)


/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_71372/261612278.py:40: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  contains_mask = big_df[big_col].str.contains(row[small_col])
